In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [5]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [6]:
result

42

In [7]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [8]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val, z_val)

10 15


# Linear Regression With TF

In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

In [3]:
housing = fetch_california_housing()

In [4]:
housing.data.shape

(20640, 8)

In [5]:
housing.target.shape

(20640,)

In [6]:
housing.target.reshape(-1,1).shape

(20640, 1)

In [7]:
m, n = housing.data.shape

In [8]:
housing.data = preprocessing.StandardScaler().fit_transform(housing.data)

In [9]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matmul(XT, X), XT), y)

In [74]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [75]:
theta_value

array([[  8.81225728e+08],
       [  3.63629632e+08],
       [  7.08457900e+06],
       [  1.51863696e+08],
       [  1.09662550e+07],
       [ -2.50988160e+07],
       [ -5.92709800e+06],
       [ -6.87514720e+07],
       [  2.85793240e+07]], dtype=float32)

# Implementing Gradient Descent

## Manually computing the Gradients

In [15]:
n_epochs = 1000
learning_rate = 0.01

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [17]:
init = tf.global_variables_initializer()

In [18]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 8.70788
Epoch 100 MSE = 0.527301
Epoch 200 MSE = 0.524392
Epoch 300 MSE = 0.524325
Epoch 400 MSE = 0.524321
Epoch 500 MSE = 0.524321
Epoch 600 MSE = 0.524321
Epoch 700 MSE = 0.524321
Epoch 800 MSE = 0.524321
Epoch 900 MSE = 0.52432


## Using an Optimizer

In [16]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)